In [118]:
import torch
from torch import nn 
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import numpy as np 
import spacy

## chats

In [155]:
chats = {'intents':
    [
        {
            'tag':'Greeting',
            'patterns':[
                'Hi',
                'Hey!',
                'Hello!',
                'Good morning',
                'Good day',
                'good evening'
            ],
            'responses':[
                'Hey! Nice to have you here. How can I help you ?',
                'Hi, how can I help you ?',
                'Hello, thank you for visiting us. How can I help you?',
                'Good day, I am here to help you.'
            ]
        },
        {
            'tag':'Bye',
            'patterns':
            [
                'Bye',
                'See you later',
                'Have a good one',
                'Have a nice day',
                'Good bye'
            ],
            'responses':[
                'You too.',
                'Have a good one.',
                'Have a nice day'
            ]
        },
        {
            'tag':'Thanks',
            'patterns':[
                'Thanks',
                'Thank you',
                "Thanks, that's helpful!"
            ],
            'responses':[
                'Happy to help you',
                'My pleasure'
            ]
        },
        {
            'tag':'Feeling',
            'patterns':[
                "I don't feel well today",
                "I am exausted, I don't know what's wrong",
                "Something wrong with me"
            ],
            'responses':[
                "Can you tell me if you have any symtom?",
                "Do you have any symtoms?",
                "What kind of symtoms do you have?"
            ]
        },
        {
            'tag':'Symtoms',
            'patterns':[
                'I have high temperature',
                'I have headache',
                'I have temperature and nausea',
                'I a dry cough and temperature'
            ],
            'responses':[
                'Okay, you should get consulted. Do you want to book an appointment?',
                'Do you want me to book an appointment for you?',
                'Symtoms can be sign of multiple things, to diagnosis we need some tests. Do you want an anppointment booked ?'
            ]
        },
        {
            'tag':'Appoinment',
            'patterns':[
                'yes please, for tomorrow',
                'yes, if possible, tomorrow',
                'for tomorrow, please',
            ],
            'responses':[
                'Okay, your appoinment is for tomorrow',
                "you can come tomorrow",
                "okay, it has been done for you!"

            ]
        }
    ]
}

In [120]:
nlp = spacy.load('en_core_web_sm')

## Preprocessing

### tokenize

In [121]:
def tokenizer(sentence):
    doc = nlp(sentence)
    tokens = [token.lemma_ for token in doc if not token.is_punct]
    return tokens

In [122]:
def prep():
   tags = []
   patterns = []
   voca = []
   pairs = []
   for intent in chats['intents']:
      tags.append(intent['tag'])
      for pattern in intent['patterns']:
         tokens = tokenizer(pattern)
         w = [token for token in tokens if not token in voca]
         voca.extend(w)
         pairs.append((tokens,intent['tag']))
   print(f'tags-{tags}')
   print(f'vocab size-{len(voca)}')
   print(f'{pairs}')
   return pairs,voca,tags



In [123]:
pairs,voca,tags = prep()

tags-['Greeting', 'Bye', 'Thanks', 'Feeling', 'Symtoms', 'Appoinment']
vocab size-44
[(['hi'], 'Greeting'), (['hey'], 'Greeting'), (['hello'], 'Greeting'), (['good', 'morning'], 'Greeting'), (['good', 'day'], 'Greeting'), (['good', 'evening'], 'Greeting'), (['bye'], 'Bye'), (['see', 'you', 'later'], 'Bye'), (['have', 'a', 'good', 'one'], 'Bye'), (['have', 'a', 'nice', 'day'], 'Bye'), (['good', 'bye'], 'Bye'), (['thank'], 'Thanks'), (['thank', 'you'], 'Thanks'), (['thank', 'that', 'be', 'helpful'], 'Thanks'), (['I', 'do', 'not', 'feel', 'well', 'today'], 'Feeling'), (['I', 'be', 'exauste', 'I', 'do', 'not', 'know', 'what', 'be', 'wrong'], 'Feeling'), (['something', 'wrong', 'with', 'I'], 'Feeling'), (['I', 'have', 'high', 'temperature'], 'Symtoms'), (['I', 'have', 'headache'], 'Symtoms'), (['I', 'have', 'temperature', 'and', 'nausea'], 'Symtoms'), (['I', 'a', 'dry', 'cough', 'and', 'temperature'], 'Symtoms'), (['yes', 'please', 'for', 'tomorrow'], 'Appoinment'), (['yes', 'if', 'possible

In [151]:
def embed(voca,sentence):
    '''
     1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    '''
    bag =np.zeros(len(voca),dtype=np.float32)
    for idx,w in enumerate(voca):
        if w in sentence:
            bag[idx] += 1
    return bag


In [125]:
def tokenize(pairs,voca,tags):
    xs = []
    labels = []
    for pair in pairs:
        # getting x and labels
        x,y = pair

        #embedding
        x = embed(voca,x)
        y = tags.index(y)
        xs.append(x)
        labels.append(y)
    return xs,labels
    

        


        

In [126]:
x,label = tokenize(pairs,voca,tags)

## dataset and dataloader

In [127]:
class CusDatset(Dataset):
    def __init__(self,xs,labels):
        self.len = len(xs)
        self.xs = xs
        self.labels = labels

    def __getitem__(self,ind):
        return self.xs[ind],self.labels[ind]
    def __len__(self):
        return self.len

In [128]:
batchsize =2
trainset = CusDatset(x,label)
trainloader = DataLoader(trainset,batch_size=batchsize,shuffle=True)

In [129]:
x,y  =  iter(trainloader).next()

## model

In [130]:
class Model(nn.Module):
    def __init__(self,in_dim,hidden1,hidden2,out_dim):
        super(Model,self).__init__()
        self.fc1 = nn.Linear(in_dim,hidden1)
        self.fc2 = nn.Linear(hidden1,hidden2)
        self.fc3 = nn.Linear(hidden2,out_dim)
    
    def forward(self,x):
        z = F.relu(self.fc1(x))
        z = F.relu(self.fc2(z))
        out = self.fc3(z)

        return out

In [131]:
input_size = len(voca)
out_size = len(tags)
model = Model(input_size,20,10,out_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.003)
epochs = 500

for epoch in range(epochs):
    losses = []
    print(f'running {epoch+1} out of {epochs} epochs')

    for x,y in trainloader:
        optimizer.zero_grad()
        y = torch.tensor(y,dtype=torch.long)
        x = torch.tensor(x,dtype=torch.float)
        out = model.forward(x)
        loss = criterion(out,y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())


    print(np.mean(losses))
torch.save(model.state_dict(),'med_bot.pth')

running 1 out of 500 epochs
1.8045118749141693
running 2 out of 500 epochs
1.7800026436646779
running 3 out of 500 epochs
1.7563117643197377
running 4 out of 500 epochs
1.7162188192208607
running 5 out of 500 epochs
1.668918530146281
running 6 out of 500 epochs
1.593627045551936
running 7 out of 500 epochs
1.488975505034129
running 8 out of 500 epochs
1.3657416999340057
running 9 out of 500 epochs
1.2304579267899196
running 10 out of 500 epochs
1.0858807439605396
running 11 out of 500 epochs
0.9469087471564611
running 12 out of 500 epochs
0.82232582817475
running 13 out of 500 epochs
0.7036415586868922
running 14 out of 500 epochs
0.5961002732316653
running 15 out of 500 epochs
0.490581420250237
running 16 out of 500 epochs
0.38751585098604363
running 17 out of 500 epochs
0.3088924443970124
running 18 out of 500 epochs
0.23705330056448778
running 19 out of 500 epochs
0.18198983774830899
running 20 out of 500 epochs
0.14210659001643458
running 21 out of 500 epochs
0.11134172743186355
ru

/var/folders/1h/x7tmv9xd1bsclx868ql32nf40000gn/T/ipykernel_44657/2523379147.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y,dtype=torch.long)
/var/folders/1h/x7tmv9xd1bsclx868ql32nf40000gn/T/ipykernel_44657/2523379147.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x,dtype=torch.float)


0.020877829131980736
running 32 out of 500 epochs
0.01861481830322494
running 33 out of 500 epochs
0.016887386368277173
running 34 out of 500 epochs
0.015182323365782699
running 35 out of 500 epochs
0.013727629596057037
running 36 out of 500 epochs
0.012744173310541859
running 37 out of 500 epochs
0.011753778846468776
running 38 out of 500 epochs
0.0108133289613761
running 39 out of 500 epochs
0.009999400392795602
running 40 out of 500 epochs
0.009254374036875864
running 41 out of 500 epochs
0.00867573160212487
running 42 out of 500 epochs
0.008102849106459567
running 43 out of 500 epochs
0.007577283094481875
running 44 out of 500 epochs
0.007170908017239223
running 45 out of 500 epochs
0.006697013314502935
running 46 out of 500 epochs
0.006321514869341627
running 47 out of 500 epochs
0.005955698269341762
running 48 out of 500 epochs
0.005669467383995652
running 49 out of 500 epochs
0.005353102809749544
running 50 out of 500 epochs
0.005070032755611464
running 51 out of 500 epochs
0.00

In [152]:
def input_prep(text,voca):
    tokens = tokenizer(text)
    x_test = embed(voca,tokens)
    return x_test


In [133]:
model = Model(input_size,20,10,out_size)
state = torch.load('med_bot.pth')
model.load_state_dict(state)

<All keys matched successfully>

## testing

In [154]:
import random
bot = 'Medbot'
print("let's have a convo, (to exit type 'quit' )")
with torch.no_grad():
    model.eval()
    while True:
        text = input('You:')
        if text == 'quit':
            print('Nice ')
            break
        x_test = input_prep(text,voca)
        x_test = torch.tensor(x_test,dtype=torch.float)
        x_test = x_test.view(1,-1)
        out = model.forward(x_test)
        _,pred = torch.max(out,1)
        tag = tags[pred.item()]
        for intent in chats['intents']:
                if tag == intent['tag']:
                    print(f'You: {text}')
                    print(f"{bot}: {random.choice(intent['responses'])}")
    





let's have a convo, (to exit type 'quit' )
You: good day
Medbot: Hello, thanks for visiting
You: I don't feel well today
Medbot: Do you have any symtoms
You: I have temperature and nausea
Medbot: Symtoms can be sign of multiple things, to diagnosis we need some tests. Do you want an anppointment booked ?
You: yes please
Medbot: okay, it has been done for you!
You: thanks
Medbot: My pleasure
